In [2]:
import pyspark

spark = pyspark.sql.SparkSession \
    .builder \
    .appName("COVID-CXR META ETL") \
    .config("spark.some.config.option", "xxx") \
    .getOrCreate()

try:
    # once intialized, it's global in the jupyter notebook kernal; one context per notebook
    sc = pyspark.SparkContext.getOrCreate('local[*]')
except:
    sc = pyspark.SparkContext('local[*]')
sqlc = pyspark.SQLContext

In [22]:
def spark_etl_META_0():
    # src 0
    rdd = spark.read.csv(INPUT_PATH_0_META, header=True, sep=",").rdd
    
    global _src0_url
    _src0_url = rdd.map(lambda x: x.url).collect()  # RDD ACTION, for duplicates in src 3
    
    rdd = rdd.map(lambda x: CXR(pid=x.patientid, fn_src=x.filename, label=x.finding, src=0))
    rdd = rdd.filter(lambda x: 1 - x.isna())
    rdd = rdd.filter(lambda x: x.view in (["PA", "AP", "AP Supine", "AP semi erect", "AP erect"]))
    rdd = rdd.filter(lambda x: x.label!= 'other')
    return rdd

def spark_etl_META_1():
    # src 1
    rdd = spark.read.csv(INPUT_PATH_1_META, header=True, sep=",").rdd
    rdd = rdd.map(lambda x: CXR(pid=x.patientid, fn_src=None, label=x.finding, src=1))
    rdd = rdd.filter(lambda x: 1- x.isna())
    rdd = rdd.filter(lambda x: x.label!= 'other')
    return rdd

def spark_etl_META_2():
    # src 2
    rdd = spark.read.csv(INPUT_PATH_2_META, header=True, sep=",").rdd
    rdd = rdd.map(lambda x: CXR(pid=x.patientid, fn_src=x.imagename, label=x.finding, src=2))
    rdd = rdd.filter(lambda x: 1- x.isna())
    rdd = rdd.filter(lambda x: x.label!= 'other')
    return rdd

def spark_etl_META_3():
    # src 3
    df0 = src3_etl(INPUT_PATH_3_0_META, 'covid')
    df1 = src3_etl(INPUT_PATH_3_1_META, 'normal')
    df2 = src3_etl(INPUT_PATH_3_2_META, 'pneumonia')
    META_3 = df0.append(df1).append(df2)
    return META_3
def spark_src3_etl(metapath, label):
    df = pd.read_csv(metapath)
    del df['SIZE']
    if label=='covid':
        # discard: # https://github.com/lindawangg/COVID-Net/blob/master/create_COVIDx.ipynb
        df = df[~df['FILE NAME'].isin(discard)]
        df = df[~df.URL.isin(_src0_url)]  # drop duplicates
    del df['URL']
    df.dropna(axis=0, how='any', inplace=True)
    df['img'] = df.apply(src3_img, axis=1)
    del df['FORMAT']
    df.rename(columns={'FILE NAME': 'patient'}, inplace=True)
    df['label'] = label
    df['src'] = 3
    return df

In [23]:
spark_etl_META_0()

PythonRDD[50] at RDD at PythonRDD.scala:53

In [39]:
from src.rdd import CXR
from src.etl import *

label = 'normal'
print(label)
rdd = spark.read.csv(INPUT_PATH_3_1_META, header=True, sep=",").rdd
if label == 'covid':
    # https://github.com/lindawangg/COVID-Net/blob/master/create_COVIDx.ipynb
    rdd = rdd.filter(lambda x: x['FILE NAME'] not in discard)
    rdd = rdd.filter(lambda x: x.URL not in _src0_url)
rdd = rdd.map(lambda x: CXR(pid=x['FILE NAME'], fn_src=f"{x['FILE NAME']}.{x.FORMAT.lower()}", label=label, src=3))
rdd = rdd.filter(lambda x: 1- x.isna())

print(rdd)
print(rdd.take(1))

normal
PythonRDD[151] at RDD at PythonRDD.scala:53


In [40]:
c = rdd.collect()

In [48]:
c[1].pid, c[1].fn_src, c[1].label, c[1].src

('NORMAL-2', 'NORMAL-2.png', 'normal', 3)

In [35]:
spark

In [83]:
for i in dir(df.map(lambda x: x.url).distinct()):
    if i[0] != '_':
        print(i)

aggregate
aggregateByKey
barrier
cache
cartesian
checkpoint
coalesce
cogroup
collect
collectAsMap
collectWithJobGroup
combineByKey
context
count
countApprox
countApproxDistinct
countByKey
countByValue
ctx
distinct
filter
first
flatMap
flatMapValues
fold
foldByKey
foreach
foreachPartition
fullOuterJoin
func
getCheckpointFile
getNumPartitions
getStorageLevel
glom
groupBy
groupByKey
groupWith
histogram
id
intersection
isCheckpointed
isEmpty
isLocallyCheckpointed
is_barrier
is_cached
is_checkpointed
join
keyBy
keys
leftOuterJoin
localCheckpoint
lookup
map
mapPartitions
mapPartitionsWithIndex
mapPartitionsWithSplit
mapValues
max
mean
meanApprox
min
name
partitionBy
partitioner
persist
pipe
preservesPartitioning
prev
randomSplit
reduce
reduceByKey
reduceByKeyLocally
repartition
repartitionAndSortWithinPartitions
rightOuterJoin
sample
sampleByKey
sampleStdev
sampleVariance
saveAsHadoopDataset
saveAsHadoopFile
saveAsNewAPIHadoopDataset
saveAsNewAPIHadoopFile
saveAsPickleFile
saveAsSequenceFile

In [19]:
sqlContext = sc.sqlContext

AttributeError: 'SparkContext' object has no attribute 'sqlContext'

## meta

In [ ]:
from src.etl import *
from src.utils import *
import yaml
from collections import Counter
from collections import defaultdict

In [ ]:
params['data']['image_size']

In [ ]:
META = etl()

In [ ]:
print('label distribution: ', Counter(META.label), sep='\n')

## train / test split

In [ ]:
META = dataset_split(META)
META.head()

In [ ]:
print('label integer map: ', params['train']['labelmap'], sep='\n', end='\n\n')
for ds in (0, 1):
    print('{0} data from covid datasets: '.format({0: 'test', 1: 'train'}[ds]), Counter(META[META.train==ds].label), sep='\n', end='\n\n')

In [ ]:
# cache image meta
if not os.path.isdir(SAVE_PATH):
    os.makedirs(SAVE_PATH)
with open(os.path.join(SAVE_PATH, 'meta'), 'wb') as pickle_file:
    pickle.dump(META, pickle_file, pickle.HIGHEST_PROTOCOL)
    print(f"META data saved at {os.path.join(SAVE_PATH, 'meta')}")

In [ ]:
%%time
if False:
    # -------------------------------------------------------------------------------------------------------------------
    # imagge file to matrix  
    # NOTE: this needs model to load all data at once into cache which take up too much storage, 
    #       making computing inefficient, NOT GOOD
    # -------------------------------------------------------------------------------------------------------------------
    # train_data / test_data structure: 
    #                        {'covid': {'data': list of size*size*3 numpy array, 'label': corresponding list of integer},
    #                        '!covid': {'data': list of size*size*3 numpy array, 'label': corresponding list of integer}}
    TRAIN_DATA, TEST_DATA = datafromfile(META)
    # a sanity check
    print(f"train covid sample #: {len(TRAIN_DATA['covid']['label'])}")
    print(f"train !covid sample #: {len(TRAIN_DATA['!covid']['label'])}")
    print(f"test  covid sample #: {len(TEST_DATA['covid']['label'])}")
    print(f"test  !covid sample #: {len(TEST_DATA['!covid']['label'])}")
    # cache image data
    with open(os.path.join(SAVE_PATH, 'train.data'), 'wb') as pickle_file:
        pickle.dump(TRAIN_DATA, pickle_file, pickle.HIGHEST_PROTOCOL)
        print(f"Image data saved at {os.path.join(SAVE_PATH, 'train.data')}")
    with open(os.path.join(SAVE_PATH, 'test.data'), 'wb') as pickle_file:
        pickle.dump(TEST_DATA, pickle_file, pickle.HIGHEST_PROTOCOL)
        print(f"Image data saved at {os.path.join(SAVE_PATH, 'test.data')}")
else:
    # merge imgaes together instead, and transform image to batches of matrix during training
    mergesoure(META)

In [ ]:
# sanity check for image shape, they need to consistent with model input
import cv2
import glob
import numpy as np
for i in np.random.choice(META.index, 10):
    print(i, cv2.imread(META.loc[i, 'imgid']).shape)